In [1]:
# Instalación de paquetes
!pip cache purge
!pip install transformers datasets sentencepiece spacy unidecode pymupdf

# Importación de librerías
import fitz  # pymupdf
import re
import spacy
import torch
from datasets import Dataset
from unidecode import unidecode
import os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'  # Establece la versión legacy
os.environ["WANDB_DISABLED"] = "true"

Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which i

In [2]:
# Descargar documento de prueba
!wget  'https://drive.google.com/uc?export=download&id=1JMHAPIZPFBnAZwOqV-lJ2T6-OHZoUlun' -O 'pdf11.pdf'  #10.FAQ_SARLAFT.pdf

# Inicializar variables y cálculo de tamaño
total_bytes = 0
nbytes = 0
for i in range(11, 12):  # antes de 1 a 18
    pdf_filename = f"pdf{i:02d}.pdf"
    nbytes = os.stat(pdf_filename).st_size
    total_bytes += nbytes
megabytes = total_bytes / 1048576
print(f"Total megabytes de todos los documentos cargados: {megabytes}")

--2024-12-13 00:30:22--  https://drive.google.com/uc?export=download&id=1JMHAPIZPFBnAZwOqV-lJ2T6-OHZoUlun
Resolving drive.google.com (drive.google.com)... 142.251.175.138, 142.251.175.102, 142.251.175.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.175.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1JMHAPIZPFBnAZwOqV-lJ2T6-OHZoUlun&export=download [following]
--2024-12-13 00:30:22--  https://drive.usercontent.google.com/download?id=1JMHAPIZPFBnAZwOqV-lJ2T6-OHZoUlun&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.12.132, 2404:6800:4003:c1c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.12.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3005487 (2.9M) [application/octet-stream]
Saving to: ‘pdf11.pdf’

pdf11.pdf           100%[===================>]   2.87M  --.-KB/s 

In [3]:
# PRE-PROCESAMIENTO
pat = re.compile(r"[^\w\s%]")
spaces = re.compile(r"\s{2,}")
def preprocess_2(text):
    text = text.replace("Internal", "")
    text = unidecode(text).lower()
    urls = re.findall(r'https?://\S+|www\.\S+', text)
    for i, url in enumerate(urls):
        text = text.replace(url, f'URL{i}')
    text = re.sub(pat, "", text)
    text = re.sub(spaces, " ", text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    for i, url in enumerate(urls):
        text = text.replace(f'URL{i}', url)
    return text

In [4]:
pdf_document = fitz.open(pdf_filename)
doc_text = ""
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    doc_text += page.get_text("text")

documento_completo = preprocess_2(doc_text)

In [5]:
questions = [
    "¿Cómo se indexa el Formato Único de Conocimiento del Cliente en la ficha de gestión del cliente?",
    "¿Si la póliza fue cancelada o anulada debe tener el Formato Único de Conocimiento del Cliente?",
    "¿Qué pasa si el cliente decidió no renovar la póliza?",
    "¿Qué pasa si la póliza fue emitida pero el cliente desistió del negocio?",
    "¿Qué pasa si después de realizar la gestión definitivamente no se consigue el Formato Único de Conocimiento del Cliente?",
    "¿Las pólizas de entidades estatales requieren SARLAFT?",
    "¿Qué pasa si el cliente cambió de intermediario?",
    "¿Qué pasa cuando el cliente se encuentra en proceso de sucesión ilíquida?",
    "¿Qué documentos se deben indexar a ficha de gestión de cliente para persona natural y persona jurídica?",
    "Una vez se indexa el Formato Único de Conocimiento del Cliente y los documentos soporte el documento de identificación a ficha de gestión de cliente, ¿A dónde se envían los documentos físicos?",
    "¿Qué debe hacer el intermediario si no usa de manera frecuente el correo corporativo @Allia2.com?",
    "¿Dónde se puede visualizar la clave de intermediario?"
]

answers = [
    {"text": "el formato unico de conocimiento del cliente se debe indexar en la ficha de gestion del cliente bajo el codigo 2612 para mayor informacion puede consultar el instructivo anexo o el video que se encuentra en el siguiente link https://vime", "answer_start": 1427},
    {"text": "si por norma todo negocio debe contar con el formato unico de conocimiento del cliente desde el momento de la emision por esta razon es necesario realizar la gestion pertinente para conseguir el documento e indexarlo en la ficha de gestion del cliente bajo el codigo 2612", "answer_start": 1798},
    {"text": "en este caso no es necesario realizar la actualizacion del documento sin embargo es obligatorio que en el sistema se encuentra indexado el formato unico de conocimiento del cliente de la primera vigencia de la poliza", "answer_start": 2122},
    {"text": "de acuerdo con las politicas y procedimientos de america es obligatorio contar con el formato unico de conocimiento del cliente para realizar la emision por esta razon es necesario realizar la gestion pertinente para conseguir el documento e indexarlo en la ficha de gestion del cliente bajo el codigo 2612", "answer_start": 2410},
    {"text": "los formatos que no se indexen en el sistema seguiran registrando como pendientes y continuaran apareciendo en las bases de datos ademas afectaran los indicadores de gestion del intermediario en relacion a este tema", "answer_start": 2836},
    {"text": "las entidades regidas bajo el derecho publico estan exentas del formato unico de conocimiento del cliente", "answer_start": 3105},
    {"text": "en este caso el nuevo intermediario debe hacer la consecucion del formato unico de conocimiento del cliente", "answer_start": 3258},
    {"text": "en estos casos se debe indexar el sarlaft de la persona que quede en cabeza del proceso de sucesion que sera el tomador de la poliza una vez definida la sucesion se debera actualizar el sarlaft con la del dueno final del bien", "answer_start": 3447},
    {"text": "persona natural formato unico de conocimiento del cliente y fotocopia de la cedula de ciudadania ampliada al 150% persona juridica formato unico de conocimiento del cliente copia del rut certificado de existencia y representacion legal emitido por la camara de comercio del domicilio de la compania no mayor a 60 dias en caso de ser entidad financiera tambien debera adjuntar el certificado de la superfinanciera fotocopia del documento de identidad del representante legal que firmo el formulario ampliada al 150% estados financieros auditados de los ultimos 2 anos en caso de tener que ampliar la informacion certificacion firmada por el representante legal o revisor fiscal con la lista de socios o accionistas que tengan una participacion directa o indirecta de mas del 5% del capital social aporte o participacion o constancia de cotizacion en bolsa de valores del cliente o del propietario que tenga mas del 5% de participacion", "answer_start": 4011},
    {"text": "una vez se indexen al sistema los documentos se deben enviar a la sucursal asignada en un sobre sellado en el cual se identifique claramente el remitente nombre del allia2 direccion y telefono y en destinatario incluir a la sucursal asignada posterior a ello la sucursal envia los documentos a back office en casa principal america carrera 13a", "answer_start": 5135},
    {"text": "para no perderse de todos los comunicados que america envia a sus allia2 el agente o agencia pueden comunicarse directamente a la linea telefonica 5188990 centro de atencion al usuario cau y solicitar redireccion automatica a un correo electronico de su preferencia", "answer_start": 5595},
    {"text": "en allia2net plus en la parte superior de la pantalla se puede visualizar el numero de la clave en el campo mediador principal en el librillo de las polizas en la segunda hoja se relacionan las condiciones particulares y se identifica el numero de clave de intermediario ", "answer_start": 5913},
]
# Verificar las posiciones de las respuestas
for i, answer in enumerate(answers):
    start_position = answer["answer_start"] - 1  # Restar 1 a la posición de inicio
    end_position = start_position + len(answer["text"])  # Ajuste para longitud de caracteres
    context = documento_completo[start_position:end_position]
    print(f"Pregunta {i+1}: {questions[i]}")
    print(f"Respuesta: {answer['text']}")
    print(f"Contexto: {context}")
    print("-" * 50)

# Inicializar el diccionario
data_dict_train = {"context": [], "question": [], "answers": []}

# Asociar todas las preguntas con el mismo contexto
for i in range(12):
    data_dict_train["context"].append(documento_completo)
    data_dict_train["question"].append(questions[i % len(questions)])
    data_dict_train["answers"].append(answers[i % len(answers)])

# Crear el dataset
dataset_train = Dataset.from_dict(data_dict_train)

# Verificar la estructura del dataset
print(dataset_train)
#
questions = [
    "¿qué significan las siglas SARLAFT?",
    "¿Por qué es importante el SARLAFT?",
    "¿Qué pasa si el cliente no tiene el SARLAFT en la ficha de gestión?",
    "¿Qué pólizas no requieren SARLAFT?",
]

answers = [
    {"text": "sistema de administración del riesgo de lavado de activos y de la financiación del terrorismo", "answer_start": 85},
    {"text": "La implementación y funcionamiento del sistema de administración del riesgo de lavado de activos y financiación del terrorismo (SARLAFT) permite a las entidades vigiladas prevenir el riesgo de ser utilizadas en operaciones con activos provenientes de actividades relacionadas con del lavado de activos y/o financiación del terrorismo", "answer_start": 212},
    {"text": "de acuerdo con lo establecido en el manual sarlaft es obligatorio que todos los clientes que tengan o hayan tenido una poliza con america seguros sa o america seguros de vida sa tengan indexado el formato unico de conocimiento del cliente en la ficha de gestion para el caso de las polizas vigentes no contar con este documento puede generar la cancelacion inmediata de las polizas que tenga el cliente no aplica para las polizas que no requieren este documento", "answer_start": 609},
    {"text": "de acuerdo con la norma las polizas de soat salud accidentes personales en vuelo los tomados por entidades financieras y las demas que se establecen en el manual sarlaft no requieren el formato unico de conocimiento del cliente", "answer_start": 1104},
]
# Verificar las posiciones de las respuestas
for i, answer in enumerate(answers):
    start_position = answer["answer_start"] - 1  # Restar 1 a la posición de inicio
    end_position = start_position + len(answer["text"])  # Ajuste para longitud de caracteres
    context = documento_completo[start_position:end_position]
    print(f"Pregunta {i+1}: {questions[i]}")
    print(f"Respuesta: {answer['text']}")
    print(f"Contexto: {context}")
    print("-" * 50)

# Inicializar el diccionario
data_dict_test = {"context": [], "question": [], "answers": []}

# Asociar todas las preguntas con el mismo contexto
for i in range(4):
    data_dict_test["context"].append(documento_completo)
    data_dict_test["question"].append(questions[i % len(questions)])
    data_dict_test["answers"].append(answers[i % len(answers)])

# Crear el dataset
dataset_test = Dataset.from_dict(data_dict_test)

# Verificar la estructura del dataset
print(dataset_test)
#

Pregunta 1: ¿Cómo se indexa el Formato Único de Conocimiento del Cliente en la ficha de gestión del cliente?
Respuesta: el formato unico de conocimiento del cliente se debe indexar en la ficha de gestion del cliente bajo el codigo 2612 para mayor informacion puede consultar el instructivo anexo o el video que se encuentra en el siguiente link https://vime
Contexto: el formato unico de conocimiento del cliente se debe indexar en la ficha de gestion del cliente bajo el codigo 2612 para mayor informacion puede consultar el instructivo anexo o el video que se encuentra en el siguiente link https://vime
--------------------------------------------------
Pregunta 2: ¿Si la póliza fue cancelada o anulada debe tener el Formato Único de Conocimiento del Cliente?
Respuesta: si por norma todo negocio debe contar con el formato unico de conocimiento del cliente desde el momento de la emision por esta razon es necesario realizar la gestion pertinente para conseguir el documento e indexarlo en la fi

In [6]:
from transformers import AutoTokenizer

#checkpoint = "bert-large-uncased-whole-word-masking-finetuned-squad"
#checkpoint = "dccuchile/bert-base-spanish-wwm-cased"
#checkpoint = "deepset/xlm-roberta-large-squad2"
checkpoint = "deepset/roberta-large-squad2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess_function(examples):
    questions = examples["question"]
    contexts = examples["context"]

    inputs = tokenizer(
        questions,
        contexts,
        truncation=True,
        max_length=512,
        stride=128,
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="np"
    )
    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []
    for i, answer in enumerate(examples["answers"]):
        start_char = answer["answer_start"]
        end_char = start_char + len(answer["text"])

        # Encuentra el token correspondiente al carácter de inicio y fin
        start_token = 0
        end_token = 0
        for idx, (start, end) in enumerate(offset_mapping[i]):
            if start_char >= start and end_char <= end:
                start_token = idx
                end_token = idx
                break

        start_positions.append(start_token)
        end_positions.append(end_token)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Tokenización de los datos
tokenized_data_train = dataset_train.map(preprocess_function, batched=True)
tokenized_data_test = dataset_test.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [7]:
import gc
gc.collect()
#
# Carga del modelo
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
# Configuración final del Trainer
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.005,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    fp16=True,
)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
# Evaluación con métricas
from transformers import AutoModelForQuestionAnswering
!pip install evaluate
from evaluate import load
accuracy_metric = load("accuracy")

# Obtener predicciones

def compute_metrics(pred):
    start_preds = pred.predictions[0].argmax(axis=1)
    end_preds = pred.predictions[1].argmax(axis=1)

    if isinstance(pred.label_ids, tuple):
        start_labels = pred.label_ids[0]
        end_labels = pred.label_ids[1]
    else:
        start_labels = pred.label_ids[:, 0]
        end_labels = pred.label_ids[:, 1]

    # Cálculo de métricas para posiciones de inicio
    em_score_start = accuracy_score(start_labels, start_preds)
    f1_start = f1_score(start_labels, start_preds, average="weighted")
    precision_start = precision_score(start_labels, start_preds, average="weighted", zero_division=0)
    recall_start = recall_score(start_labels, start_preds, average="weighted", zero_division=0)

    # Cálculo de métricas para posiciones de fin
    em_score_end = accuracy_score(end_labels, end_preds)
    f1_end = f1_score(end_labels, end_preds, average="weighted")
    precision_end = precision_score(end_labels, end_preds, average="weighted", zero_division=0)
    recall_end = recall_score(end_labels, end_preds, average="weighted", zero_division=0)

    # Promedio de las métricas
    em_score = (em_score_start + em_score_end) / 2
    f1 = (f1_start + f1_end) / 2
    precision = (precision_start + precision_end) / 2
    recall = (recall_start + recall_end) / 2

    return {
        "Exact Match": em_score,
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall
    }




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data_train,
    eval_dataset=tokenized_data_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenamiento y evaluación
trainer.train()
results = trainer.evaluate()
print(results)

<ipython-input-9-1f6905e49771>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Exact match,F1 score,Precision,Recall
1,No log,0.785217,1.000000,1.000000,1.000000,1.000000
2,No log,0.785217,1.000000,1.000000,1.000000,1.000000
3,No log,0.785217,1.000000,1.000000,1.000000,1.000000


{'eval_loss': 0.78521728515625, 'eval_Exact Match': 1.0, 'eval_F1 Score': 1.0, 'eval_Precision': 1.0, 'eval_Recall': 1.0, 'eval_runtime': 0.1414, 'eval_samples_per_second': 28.288, 'eval_steps_per_second': 7.072, 'epoch': 3.0}
